<a href="https://colab.research.google.com/github/himoyuzuki/colab_analytics/blob/main/%E6%99%AF%E6%B0%97%E5%8B%95%E5%90%91%E6%8C%87%E6%95%B0%E3%81%8B%E3%82%89%E8%A6%8B%E3%82%8B%E6%97%A5%E6%9C%AC%E3%81%AE%E6%99%AF%E6%B0%97%E5%8B%95%E5%90%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 景気動向指数から見る日本の景気動向

日本経済を分析しようと思ったときアプローチは様々考えられるが、消費者として経済に関わっていることを実感でき、かつ定量的な指標がないかと思った。
<br>
政府統計である[e-Stat](https://www.e-stat.go.jp/)を眺めてみたところ、景気動向指数が良さそうだったので調べてみることにする。
<br>

- [景気動向指数 : 経済社会総合研究所 - 内閣府](https://www.esri.cao.go.jp/jp/stat/di/menu_di.html)

<br>
次のセクションで景気動向指数について詳細をまとめるが、今回の分析では先行指標を中心に分析し、予測できないか検証する。


## 景気動向指数とは

> 景気動向指数は、生産、雇用など様々な経済活動での重要かつ景気に敏感に反応する指標の動きを統合することによって、 景気の現状把握及び将来予測に資するために作成された指標です。

### 構成

以下2つで構成される。CIの変化幅そのものからは各経済部門への波及度合いの相違を把握することが難しいため、CIの変化幅に対する各採用系列の寄与度やDIをあわせて利用するのが望ましい。
- コンポジット・インデックス（CI）
  - 構成する指標の動きを合成することで景気変動の大きさやテンポ（量感）を測定する
  - 2008年以降はこちらが中心的
  - 上昇しているときは景気の拡張局面、低下しているときは後退局面
  - 変化の大きさから、景気の拡張または後退のテンポを読み取る。つまりある程度の期間の持続性が重要
  - 単月での一時的低下などもあるため、3ヶ月後方移動平均と7ヶ月後方移動平均も合わせて確認が望ましい

- ディフュージョン・インデックス（DI）
  - 構成する指標のうち、改善している指標の割合を算出することで景気の各経済部門への波及の度合い（波及度）を測定する
  - 景気拡張局面では50%を上回り、後退局面では下回る傾向がある
  - ヒストリカルDIはまた異なる指標である

さらに、それぞれに対して景気に対し先行して動く先行指数（数ヶ月先行）、ほぼ一致して動く一致指数、遅れて動く遅行指数（数ヶ月から半年程度）の3つの指数がある

### 採用している指標
- 先行指数  11
- 一致指数  10
- 遅行指数  9

## 使用データ
- [統計表一覧：景気動向指数 結果 : 経済社会総合研究所 - 内閣府](https://www.esri.cao.go.jp/jp/stat/di/di.html)
- 系列 （令和6(2024)年9月27日） 長期系列(CI指数、DI指数、DI累積指数)(Excel形式：58KB)

**データについて**
- 毎月上旬ごろ、その前々月の速報値が公開され、月末に改定状況が公開される
- 遡及して改正されることもあるため、ちょくちょく更新がある
- 基本的には上記サイトの「速報資料」を見れば政府による分析、見解が見られる（<s>わざわざColaboratoryで分析する必要はない</s>）
- 速報資料には先行指標採用系列の寄与度や個別系列の数値なども記載されているが、なぜかExcelファイルには存在しない
- また速報資料には各指標のグラフと算出方法について掲載されているものの、特に考察などはされていないため「我々は統計という事実をお伝えしているだけです」というスタンスの模様

In [ ]:
%pip uninstall numpy -y
%pip install numpy==1.26.4

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 41.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.2 MB/s eta 0:00:00
^C


※numpyのバージョンを下げるため、一度ランタイムが切断される。再起動し、以下のセルから実行すること

In [1]:
%pip install pmdarima

  Using cached pmdarima-2.0.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_28_x86_64.whl.metadata (7.8 kB)
Using cached pmdarima-2.0.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_28_x86_64.whl (2.2 MB)


In [2]:
%pip list | grep numpy

numpy                                 1.26.4


numpyのバージョンが1.26.4ならOK

In [40]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from statsmodels.tsa.stattools import adfuller
from pmdarima import auto_arima
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

In [27]:
# データの読み込み
# 注）URLは毎月変更されるため、古くなっている可能性あり。以下で最新の「長期系列」に存在するExcelファイルを確認すること
# https://www.esri.cao.go.jp/jp/stat/di/di.html
df = pd.read_excel('https://www.esri.cao.go.jp/jp/stat/di/0707ci.xlsx')

# カラム名の変更と年月カラムの作成
row_at_index_1 = df.iloc[1]
new_columns = [row_at_index_1[col] if col.startswith('Unnamed') else f'{col}_{row_at_index_1[col]}' for col in df.columns]
df.columns = new_columns
df['年月'] = df['西暦年'].astype(str) + df['月'].astype(str).str.zfill(2)
df = df.dropna(subset=['年月'])
df = df[df['年月'].str.len() == 6]
df = df[df['年月'].str.isnumeric()]
df['年月'] = pd.to_datetime(df['年月'], format='%Y%m')

df['3ヶ月後方移動平均'] = df['ＣＩ指数_先行指数'].rolling(window=3).mean()
df['7ヶ月後方移動平均'] = df['ＣＩ指数_先行指数'].rolling(window=7).mean()
df['12ヶ月後方移動平均'] = df['ＣＩ指数_先行指数'].rolling(window=12).mean()

In [28]:
df.tail()

,時間軸コード,西暦年,月,ＣＩ指数_先行指数,ＣＩ指数.1_一致指数,ＣＩ指数.2_遅行指数,(参考)「外れ値」処理なしＣＩ指数_先行指数,(参考)「外れ値」処理なしＣＩ指数.1_一致指数,(参考)「外れ値」処理なしＣＩ指数.2_遅行指数,ＤＩ指数_先行指数,ＤＩ指数.1_一致指数,ＤＩ指数.2_遅行指数,ＤＩ累積指数_先行指数,ＤＩ累積指数.1_一致指数,ＤＩ累積指数.2_遅行指数,年月,3ヶ月後方移動平均,7ヶ月後方移動平均,12ヶ月後方移動平均
545,2025000101,2025,1,108.1,116.3,111.2,109.1,114.4,109.4,54.5,60,77.8,1262.7,3280,3168.5,2025-01-01,107.900000,108.228571,109.391667
546,2025000202,2025,2,107.8,117.1,111,108.8,115.4,109.1,50,65,77.8,1262.7,3295,3196.3,2025-02-01,107.933333,108.028571,109.058333
547,2025000303,2025,3,107.6,115.8,111,109.9,114.2,109,54.5,40,66.7,1267.2,3285,3213,2025-03-01,107.833333,108.057143,108.700000
548,2025000404,2025,4,104.2,116,112.5,103.5,114.4,110.5,10,38.9,43.8,1227.2,3273.9,3206.8,2025-04-01,106.533333,107.428571,108.133333
549,2025000505,2025,5,105.3,115.9,112.7,104.2,114.6,110.6,22.2,18.8,25,1199.4,3242.7,3181.8,2025-05-01,105.700000,106.942857,107.666667


## グラフの描画

In [29]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['年月'], y=df['ＣＩ指数_先行指数'], mode='lines', name='CI先行指数', line=dict(color='red', width=2)))
fig.add_trace(go.Scatter(x=df['年月'], y=df['3ヶ月後方移動平均'], mode='lines', name='3ヶ月後方移動平均', line=dict(color='green', width=1)))
fig.add_trace(go.Scatter(x=df['年月'], y=df['7ヶ月後方移動平均'], mode='lines', name='7ヶ月後方移動平均', line=dict(color='blue', width=0.5)))

fig.update_layout(
    title='CI先行指数の推移',
    xaxis_title='年月',
    yaxis_title='CI先行指数',
    yaxis=dict(range=[90, 125]),
    xaxis=dict(range=[pd.to_datetime('2021-01-01'), df['年月'].max()]),
    hovermode='x unified',
    width=800,
    height=400,
    legend=dict(
      orientation="h",
      yanchor="bottom",
      y=1.02,
      xanchor="right",
      x=1
    )
)

fig.show()

In [30]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['年月'], y=df['ＣＩ指数_先行指数'], mode='lines', name='CI先行指数', line=dict(color='red', width=2)))
fig.add_trace(go.Scatter(x=df['年月'], y=df['3ヶ月後方移動平均'], mode='lines', name='3ヶ月後方移動平均', line=dict(color='green', width=1)))
fig.add_trace(go.Scatter(x=df['年月'], y=df['7ヶ月後方移動平均'], mode='lines', name='7ヶ月後方移動平均', line=dict(color='blue', width=0.5)))
fig.add_trace(go.Scatter(x=df['年月'], y=df['12ヶ月後方移動平均'], mode='lines', name='12ヶ月後方移動平均', line=dict(color='red', width=0.5)))

fig.update_layout(
    title='CI先行指数の全体推移',
    xaxis_title='年月',
    yaxis_title='CI先行指数',
    yaxis=dict(range=[70, 140]),
    hovermode='x unified',
    width=800,
    height=400,
    legend=dict(
      orientation="h",  # 水平方向に凡例を配置
      yanchor="bottom", # 凡例の上端を基準にする
      y=1.02,           # グラフの上部に配置 (調整可能)
      xanchor="right",  # 凡例の右端を基準にする
      x=1               # グラフの右に配置 (調整可能)
    )
)

fig.show()

## 線形回帰モデルによる予測

In [31]:
# 予測モデルに使用する特徴量と目的変数を設定
df_cleaned = df.dropna(subset=['ＣＩ指数_先行指数']) # NaNがある行を削除した新しいDataFrameを作成
X_cleaned = df_cleaned.index.values.reshape(-1, 1)  # クリーニング後のindexを特徴量として使用
y_cleaned = df_cleaned['ＣＩ指数_先行指数']

# データを訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X_cleaned, y_cleaned, test_size=0.2, random_state=42)

# 線形回帰モデルを作成
model = LinearRegression()

# モデルを訓練データで学習
model.fit(X_train, y_train)

# テストデータを用いて予測
y_pred = model.predict(X_test)

# 将来値を予測 (クリーニング後のDataFrameの最後のindexから計算)
future_X = [[df_cleaned.index[-1] + i] for i in range(1, 4)] # 予測したい将来のindexを指定 (3ヶ月分)
future_y_pred = model.predict(future_X)

# 予測結果を出力
print("将来値の予測:")
for i, prediction in enumerate(future_y_pred):
  print(f" {i+1}か月後: {prediction:.2f}")

将来値の予測:
 1か月後: 115.85
 2か月後: 115.90
 3か月後: 115.95


In [32]:
fig = go.Figure()

# 実際の値のプロット
fig.add_trace(go.Scatter(x=df_cleaned['年月'], y=df_cleaned['ＣＩ指数_先行指数'], mode='lines', name='実際の値', line=dict(color='blue')))

# 予測値のプロット
# X_testは元のindexなので、それを使って年月を取得
test_dates = df_cleaned.loc[X_test.flatten(), '年月']
fig.add_trace(go.Scatter(x=test_dates, y=y_pred, mode='lines', name='予測値 (テストデータ)', line=dict(color='pink')))

# 将来値予測のプロット (赤色の線)
# 将来の年月を生成
last_date = df_cleaned['年月'].iloc[-1]
future_dates = [last_date + pd.DateOffset(months=i) for i in range(1, len(future_y_pred) + 1)]
fig.add_trace(go.Scatter(x=future_dates, y=future_y_pred, mode='lines', name='将来値予測', line=dict(color='red')))


fig.update_layout(
    title='CI先行指数予測',
    xaxis_title='年月',
    yaxis_title='CI先行指数',
    hovermode='x unified',
    xaxis=dict(range=[pd.to_datetime('2021-01-01'), future_dates[-1]]) # x軸の範囲を将来予測の最後まで含める
)

fig.show()

### 考察

単純な線形回帰による分析では、全体の平均値に基づく線形変化しか捉えきれず、近年の現象トレンドを的確に反映できていない。

## ARIMAモデルによる予測

In [33]:
df_ts = df[['年月', 'ＣＩ指数_先行指数']].copy()
df_ts = df_ts.set_index('年月')
df_ts.dropna(inplace=True)

display(df_ts.head())

,ＣＩ指数_先行指数
年月,
1985-01-01,84.6
1985-02-01,84.9
1985-03-01,84.8
1985-04-01,85.4
1985-05-01,85.4


In [34]:
# Perform ADF test
result = adfuller(df_ts['ＣＩ指数_先行指数'])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistic: -3.007347
p-value: 0.034214
Critical Values:
	1%: -3.444
	5%: -2.868
	10%: -2.570


ARIMAモデルの次数(p, d, q)を選択します。自己相関関数(ACF)や偏自己相関関数(PACF)のグラフ、または`auto_arima`のような自動選択ツールを使用して最適な次数を決定します。


In [35]:
# ARIMAモデルの次数選択
optimal_model = auto_arima(df_ts['ＣＩ指数_先行指数'],
                           start_p=0, start_q=0,
                           max_p=5, max_q=5,
                           d=0,
                           seasonal=False,
                           stepwise=True,
                           suppress_warnings=True,
                           trace=True)

print(optimal_model.summary())

optimal_order = optimal_model.order
print(f"\nOptimal ARIMA order (p, d, q): {optimal_order}")

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=5875.379, Time=0.04 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=inf, Time=0.16 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=inf, Time=0.16 sec


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



 ARIMA(1,0,1)(0,0,0)[0]             : AIC=1649.658, Time=0.23 sec


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



...

 ARIMA(0,0,5)(0,0,0)[0] intercept   : AIC=2018.107, Time=0.87 sec


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



 ARIMA(2,0,3)(0,0,0)[0] intercept   : AIC=1622.705, Time=0.71 sec


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



 ARIMA(2,0,5)(0,0,0)[0] intercept   : AIC=1624.646, Time=1.74 sec

Best model:  ARIMA(1,0,4)(0,0,0)[0] intercept
Total fit time: 20.367 seconds
                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  485
Model:               SARIMAX(1, 0, 4)   Log Likelihood                -803.851
Date:                Thu, 17 Jul 2025   AIC                           1621.703
Time:                        23:12:23   BIC                           1650.992
Sample:                    01-01-1985   HQIC                          1633.211
                         - 05-01-2025                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      2.5585      1.155      2.215      0.027       0.294

結果: Optimal ARIMA order (p, d, q): (1, 0, 4)

In [36]:
y_train_numeric = pd.to_numeric(y_train, errors='coerce')
y_train_numeric = y_train_numeric.interpolate(method='linear')

# ARIMAモデルの訓練
arima_model = ARIMA(y_train_numeric, order=optimal_order)
arima_model_fitted = arima_model.fit()

print(arima_model_fitted.summary())

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.



                               SARIMAX Results                                
Dep. Variable:              ＣＩ指数_先行指数   No. Observations:                  388
Model:                 ARIMA(1, 0, 4)   Log Likelihood               -1455.139
Date:                Thu, 17 Jul 2025   AIC                           2924.278
Time:                        23:12:28   BIC                           2952.005
Sample:                             0   HQIC                          2935.272
                                - 388                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        102.7845      0.480    214.160      0.000     101.844     103.725
ar.L1         -0.4333      0.645     -0.672      0.502      -1.697       0.831
ma.L1          0.3513      0.650      0.540      0.5

In [37]:
# モデルによる予測

# Get the start and end indices for the test set predictions
start_index_test = X_test.min()
end_index_test = X_test.max()

# Generate predictions for the test data period
# Use the original index values from the cleaned dataframe for prediction indices
test_predictions = arima_model_fitted.predict(start=start_index_test, end=end_index_test)

# Generate future predictions (e.g., next 3 months)
# The number of steps to predict into the future
n_future_steps = 3
future_predictions = arima_model_fitted.predict(start=len(y_train_numeric), end=len(y_train_numeric) + n_future_steps - 1)

# Display the test predictions and future predictions
print("Test Data Predictions:")
print(test_predictions)

print("\nFuture Predictions:")
print(future_predictions)

Test Data Predictions:
65     101.882174
66     103.519618
67     103.228828
68     103.707107
69     101.654163
          ...    
541    102.784505
542    102.784505
543    102.784505
544    102.784505
545    102.784505
Name: predicted_mean, Length: 481, dtype: float64

Future Predictions:
388    103.097084
389    103.784959
390    103.977140
Name: predicted_mean, dtype: float64


/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.



In [38]:
# 結果の可視化
fig = go.Figure()

# Add trace for the actual values
fig.add_trace(go.Scatter(x=df_cleaned['年月'], y=df_cleaned['ＣＩ指数_先行指数'], mode='lines', name='実際の値', line=dict(color='blue')))

# Add trace for the test data predictions
# Ensure test_predictions are aligned with the actual test data dates
# Get the dates corresponding to the test predictions' index
test_dates_for_plot = df_cleaned.loc[test_predictions.index, '年月']
# Create a Series for plotting test predictions with the correct dates as index
test_predictions_series = pd.Series(test_predictions.values, index=test_dates_for_plot)

fig.add_trace(go.Scatter(x=test_predictions_series.index, y=test_predictions_series.values, mode='lines', name='予測値 (テストデータ)', line=dict(color='pink')))

# Generate future dates for the future predictions
last_date = df_cleaned['年月'].iloc[-1]
future_dates = [last_date + pd.DateOffset(months=i) for i in range(1, len(future_predictions) + 1)]

# Add trace for the future predictions
fig.add_trace(go.Scatter(x=future_dates, y=future_predictions.values, mode='lines', name='将来値予測', line=dict(color='red')))

# Update layout
fig.update_layout(
    title='CI先行指数予測 (ARIMAモデル)',
    xaxis_title='年月',
    yaxis_title='CI先行指数',
    hovermode='x unified',
    xaxis=dict(range=[pd.to_datetime('2021-01-01'), future_dates[-1]])
)

# Display the figure
fig.show()

In [41]:
# モデルの検証

# Ensure y_test is numeric
y_test_numeric = pd.to_numeric(y_test, errors='coerce')

# Ensure test_predictions is aligned with y_test and numeric
# Align test_predictions with y_test based on index
test_predictions_aligned = test_predictions.reindex(y_test_numeric.index)

# Drop any NaNs that might have resulted from reindexing if there were index mismatches
# (Although with X_test from df_cleaned index, this should not be an issue, it's safer)
combined = pd.DataFrame({'actual': y_test_numeric, 'predicted': test_predictions_aligned}).dropna()
y_test_numeric_cleaned = combined['actual']
test_predictions_aligned_cleaned = combined['predicted']


# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test_numeric_cleaned, test_predictions_aligned_cleaned))
print(f"Root Mean Squared Error (RMSE) on the test set: {rmse:.2f}")

# Re-display the plot comparing actual vs predicted on the test set for visual evaluation
fig = go.Figure()

# Add trace for the actual values (zoomed to test period)
fig.add_trace(go.Scatter(x=df_cleaned.loc[y_test_numeric_cleaned.index, '年月'], y=y_test_numeric_cleaned, mode='markers', name='実際の値 (テストデータ)', line=dict(color='blue')))

# Add trace for the test data predictions (zoomed to test period)
fig.add_trace(go.Scatter(x=df_cleaned.loc[test_predictions_aligned_cleaned.index, '年月'], y=test_predictions_aligned_cleaned, mode='markers', name='予測値 (テストデータ)', line=dict(color='pink')))


# Update layout
fig.update_layout(
    title='CI先行指数予測 vs 実際の値 (テストデータ期間)',
    xaxis_title='年月',
    yaxis_title='CI先行指数',
    hovermode='x unified',
    xaxis=dict(range=[df_cleaned.loc[y_test_numeric_cleaned.index, '年月'].min(), df_cleaned.loc[y_test_numeric_cleaned.index, '年月'].max()]) # Set x-axis range to the test period
)

# Display the figure
fig.show()

Root Mean Squared Error (RMSE) on the test set: 9.54


### Summary:

#### Data Analysis Key Findings

*   The `pmdarima` library was successfully installed for implementing the ARIMA model.
*   Missing values in the 'ＣＩ指数_先行指数' column were handled using linear interpolation.
*   The Augmented Dickey-Fuller (ADF) test indicated that the time series data is likely stationary (p-value = 0.016904), so differencing was not required.
*   An attempt to automatically determine the optimal ARIMA model orders using `pmdarima.auto_arima` failed due to a persistent `ValueError` related to numpy compatibility.
*   Despite the `auto_arima` failure, an ARIMA model with a manually specified order (5, 0, 0) was successfully trained on the numeric training data.
*   Predictions were successfully generated for both the test data period and a future period (3 months) using the trained ARIMA model, although a warning about the unsupported index preventing forecasting was noted.
*   The Root Mean Squared Error (RMSE) on the test set was calculated as 10.35.
*   A visualization comparing the actual values, test data predictions, and future predictions showed that the ARIMA model captures some trend but struggles with short-term fluctuations and rapid changes.

#### Insights or Next Steps

*   To improve prediction accuracy, consider incorporating external factors relevant to the CI leading index (e.g., economic indicators, policy changes) using models like SARIMAX or exploring non-linear models such as LSTMs.


### 考察

景気が今後数ヶ月で「悪くなる」という予想になった。線形回帰モデルに比べると非常に良い。ただし、予測値の分布を見ると実際の分布よりも平均に回帰している傾向も見られるため、単一指標の予測はこの辺りが精度として限界な印象。